# Нахождение меток для data/posts.xls

Для определения меток исходного датасета мы будем использовать
 
предобученную модель  [DeepPavlov/rubert-base-cased](https://huggingface.co/cointegrated/rubert-base-cased-nli-threeway?)



In [22]:
! pip install transformers sentencepiece --quiet
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_checkpoint = 'cointegrated/rubert-base-cased-nli-threeway'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint)


[notice] A new release of pip available: 22.3 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [23]:
def predict_zero_shot(text, label_texts, model, tokenizer, label='entailment', normalize=True):
    label_texts
    tokens = tokenizer([text] * len(label_texts), label_texts, truncation=True, return_tensors='pt', padding=True)
    with torch.inference_mode():
        result = torch.softmax(model(**tokens.to(model.device)).logits, -1)
    proba = result[:, model.config.label2id[label]].cpu().numpy()
    if normalize:
        proba /= sum(proba)
    return proba

In [26]:
import pandas as pd


data = pd.read_excel('../data/posts.xlsx')   

classes = ['Финансы', 'Технологии', 'Политика', 
           'Шоубиз', 'Fashion', 'Крипта', 'Путешествия/релокация', 
           'Образовательный контент', 'Развлечения', 'Общее']

In [ ]:
from numpy import argmax
from tqdm import tqdm

labels = {}

for indx, paper in enumerate(tqdm(data['text'][:10000])):
    proba = predict_zero_shot(paper, classes, model, tokenizer)
    labels[indx] = classes[argmax(proba)]


for i in range(10):
    print(labels[i], '|', data['text'][i])